In [7]:
#@title Starting Cell

import pandas as pd
import numpy as np
import json, urllib
from pathlib import Path
import sklearn.model_selection as skms
import multiprocessing
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import GridSearchCV
import sklearn.model_selection as skms
import xgboost as xgb
import math
# import geopy
from geopy.distance import distance as geodesic
from PIL import Image
import requests
from io import BytesIO
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from scipy.special import expit as sigmoid, logit as inverse_sigmoid
from copy import deepcopy
from scipy import stats
import statsmodels.api as sm
import altair as alt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder


In [8]:
#@title Initialization

# Define the scaler
scaler = MinMaxScaler()

# Load the CSV files into a pandas DataFrame
train_data = pd.read_csv('https://raw.githubusercontent.com/phi1ny3/raw-csvs/main/grain-training.csv')
holdout_data = pd.read_csv('https://raw.githubusercontent.com/phi1ny3/raw-csvs/main/grain-holdout.csv')


# Define X_holdout
holdout_data = holdout_data[['Area', 'Perimeter', 'Major_Axis_Length', 'Minor_Axis_Length', 'Eccentricity', 'Convex_Area', 'Extent']]
X_holdout = holdout_data.to_numpy()

holdout_data.head()

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent
0,12899,472.263000,205.520554,80.433365,0.920236,13107,0.585919
1,11693,424.503998,176.024063,84.991318,0.875709,11842,0.606735
2,11329,419.404999,173.390457,84.345825,0.873708,11560,0.803247
3,12278,453.692993,184.050156,86.660088,0.882213,12601,0.629835
4,11231,427.666992,183.587097,78.861732,0.903038,11427,0.721184


In [9]:
#@title Cleaning

# Create a LabelEncoder object
le = LabelEncoder()

# Define X_train and y_train
X_train = train_data.drop('Class', axis=1)
y_train = train_data['Class'].values

# Clean the training data
X_train = X_train.drop('Unnamed: 0', errors='ignore')
X_train = X_train.dropna()
X_train = X_train.drop_duplicates()

# Define the scaler and scale the training data
X_train_scaled = scaler.fit_transform(X_train)

# Convert the target variable to numerical values
y_train_encoded = le.fit_transform(y_train)
y_encoded = le.fit_transform(train_data['Class'])
y_pred = 

# Create a DataFrame that contains the predicted target values
predictions_df = pd.DataFrame(data=y_pred, columns=['Class'])


print(X_train.shape)
print(X_holdout.shape)
print(y_train.shape)
print(y_train_encoded.shape)

# Scale the holdout data
X_holdout_scaled = scaler.transform(X_holdout)





NameError: ignored

In [ ]:
#@title Chart it - Before

# Set the style of the plots
sns.set_style('darkgrid')

# Create a pairplot to visualize the relationship between variables and classes
sns.pairplot(train_data, hue='Class')

# Display the plot
plt.show()

# Create a scatter plot to visualize the relationship between area and class
sns.scatterplot(data=train_data, x='Area', y='Perimeter', hue='Class')

# Display the plot
plt.show()

# Create a box plot to visualize the distribution of major axis length for each class
sns.boxplot(data=train_data, x='Class', y='Major_Axis_Length')

# Display the plot
plt.show()

In [ ]:
#@title Test-train split

# Test-train split
X_train_scaled, X_test_scaled, y_train_encoded, y_test_encoded = train_test_split(X_train_scaled, y_train_encoded, test_size=0.2, random_state=42, stratify=y_train_encoded)

# Print the shapes of the train and test sets
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"y_train_encoded shape: {y_train_encoded.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")
print(f"y_test_encoded shape: {y_test_encoded.shape}")

# Display the number of samples in each set
print(f'Training set: {X_train_scaled.shape[0]} samples')
print(f'Testing set: {X_test_scaled.shape[0]} samples')


In [ ]:
#@title Training Random Forest
# Define the model
rf_model = RandomForestRegressor(random_state=42)
df_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
print(df_train_scaled.columns)
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# Perform a grid search to find the best parameters
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search_rf.fit(X_train_scaled, y_train_encoded)

# Now for test data
grid_search_rf.fit(X_test_scaled, y_test_encoded)

# Print the best parameters and the corresponding score
print("Best parameters: ", grid_search_rf.best_params_)
print("Best score: ", grid_search_rf.best_score_)

In [ ]:
#@title Training Gradient Boost

# Define the model
gb_model = GradientBoostingClassifier()

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# Perform a grid search to find the best parameters
grid_search_gb = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search_gb.fit(X_train, y_train)

# Now for testing data
grid_search_gb.fit(X_test_scaled, y_test_encoded)

# Print the best parameters and the corresponding score
print("Best parameters: ", grid_search_gb.best_params_)
print("Best score: ", grid_search_gb.best_score_)

In [ ]:
# Predict using the best model
rf_best = grid_search_rf.best_estimator_
gb_best = grid_search_gb.best_estimator_

# Make predictions on the holdout set using the best models
rf_holdout_preds = rf_best.predict(X_holdout_scaled)
gb_holdout_preds = gb_best.predict(X_holdout)

# Print the predictions
print("Random Forest Predictions: ", rf_holdout_preds)
print("Gradient Boosting Predictions: ", gb_holdout_preds)

In [ ]:
from google.colab import files
# Add the predicted target values as a new column to the test data DataFrame
y_test_encoded= predictions_df

# Save the DataFrame to a CSV file
y_test_encoded.to_csv('predictions.csv', index=False)

# turns out, gradient boost consistently outperformed random forest, so we'll output with gradient boost predictions
# Make predictions with the best model (either Random Forest or Gradient Boosting)
if grid_search_rf.best_score_ > grid_search_gb.best_score_:
    y_pred = grid_search_rf.best_estimator_.predict(X_holdout_scaled)
else:
    y_pred = grid_search_gb.best_estimator_.predict(X_holdout_scaled)

# Create a pandas dataframe with the predicted labels
results_df = pd.DataFrame({'Class': gb_holdout_preds})

# Output the results to a csv file
if grid_search_rf.best_score_ > grid_search_gb.best_score_:
    results_df.to_csv('random_forest_predictions.csv', index=False)
    files.download('random_forest_predictions.csv')
else:
    results_df.to_csv('gradient_boost_predictions.csv', index=False)
    files.download('gradient_boost_predictions.csv')

